In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [3]:
fileslist = glob('datasets/*.csv')
fileslist.sort()

In [4]:
# computing largest contiguous segment
fpath = fileslist[0]
df = pd.read_csv(fpath, index_col=[0], parse_dates=True)
df

,Monitor_01
timestamp_round,
2018-03-01 00:00:00,36.953389
2018-03-01 00:15:00,38.771572
2018-03-01 00:30:00,42.463675
2018-03-01 00:45:00,45.901771
2018-03-01 01:00:00,46.963135
2018-03-01 01:15:00,49.412439
2018-03-01 01:30:00,53.902829
2018-03-01 01:45:00,54.909765
2018-03-01 02:00:00,58.012218


In [16]:
index_start_final = index_end_final = None
count_final = 0
index_start = index_end = None
count = 0
state = False
for tup in df.itertuples():
    if ~np.isnan(tup[1:]).any():
        count += 1
        if state == False:
            # entering a contiguous region
            state = True
            index_start = tup[0]
            index_end = tup[0]
        else:
            # already in a contiguous region
            index_end = tup[0]
    else:
        if state == True:
            # end of a contiguous region
            state = False
            if count > count_final:
                count_final = count
                index_start_final = index_start
                index_end_final = index_end
                count = 0
                index_start = index_end = None

In [17]:
index_start_final, index_end_final, count_final

(Timestamp('2018-03-22 12:45:00'), Timestamp('2018-07-09 03:00:00'), 10426)

In [18]:
index_start_final.isoformat()

'2018-03-22T12:45:00'

In [19]:
import os

fout = open('kNN_availability.csv', 'w')
for fpath in fileslist:
    df = pd.read_csv(fpath, index_col=[0], parse_dates=True)
    index_start_final = index_end_final = None
    count_final = 0
    index_start = index_end = None
    count = 0
    state = False
    for tup in df.itertuples():
        if ~np.isnan(tup[1:]).any():
            count += 1
            if state == False:
                # entering a contiguous region
                state = True
                index_start = tup[0]
                index_end = tup[0]
            else:
                # already in a contiguous region
                index_end = tup[0]
        else:
            if state == True:
                # end of a contiguous region
                state = False
                if count > count_final:
                    count_final = count
                    index_start_final = index_start
                    index_end_final = index_end
                    count = 0
                    index_start = index_end = None
    fout.write('{},{},{},{}\n'.format(os.path.split(fpath)[1][:-4], index_start_final.isoformat(), index_end_final.isoformat(), count_final))
fout.close()